# Load and Visualize Code

In [1]:
import open3d as o3d
import numpy as np
import time
import os
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Path to KITTI .bin file
file_path = "kitti/2011_09_26/0005/velodyne_points/data/0000000000.bin"

# Load .bin file (x, y, z, intensity)
points = np.fromfile(file_path, dtype=np.float32).reshape(-1,4)

# Create Open3D point cloud (use x,y,z)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])

# Visualize
o3d.visualization.draw_geometries([pcd], window_name="Raw Point Cloud")

# Print stats
print("Number of points:", len(points))

Number of points: 123397


- **Raw Point Cloud**  
    ![Raw Point Cloud](raw.png)

# Exploratory Tasks

## Print point cloud stats

In [3]:
print("Number of points:", len(np.asarray(pcd.points)))
print("Sample points:", np.asarray(pcd.points)[:5])  #First 5 points
print("Intensity range:", points[:, 3].min(), points[:, 3].max())

Number of points: 123397
Sample points: [[22.71899986  0.031       0.977     ]
 [18.05200005  0.076       0.81999999]
 [18.02599907  0.132       0.81900001]
 [23.03199959  0.249       0.98799998]
 [23.13899994  0.32300001  0.99199998]]
Intensity range: 0.0 0.99


## Load another frame

In [4]:
points2 = np.fromfile("kitti/2011_09_26/0005/velodyne_points/data/0000000001.bin", dtype=np.float32).reshape(-1, 4)
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points2[:, :3])
o3d.visualization.draw_geometries([pcd2], window_name="Frame 1")

# Downsample the Point Cloud

## Code Downsampling

In [5]:
# Downsample with voxel grid
voxel_size = 0.1  # 10 cm
pcd_down = pcd.voxel_down_sample(voxel_size)

# Visualize
o3d.visualization.draw_geometries([pcd_down], window_name="Downsampled Point Cloud")

# Print stats
print("Downsampled points:", len(np.asarray(pcd_down.points)))

Downsampled points: 62162


- **Downsampled Point Cloud**  
![Downsampled Point Cloud](downsampled.png)

## Exploratory Tasks

In [6]:
print("Original points:", len(np.asarray(pcd.points)))
print("Downsampled points voxel 0.1:", len(np.asarray(pcd_down.points)))

Original points: 123397
Downsampled points voxel 0.1: 62162


## Test voxel sizes (0.05, 0.2)

In [7]:
pcd_down_05 = pcd.voxel_down_sample(voxel_size=0.05)
pcd_down_20 = pcd.voxel_down_sample(voxel_size=0.2)
o3d.visualization.draw_geometries([pcd_down_05], window_name="Voxel 0.05")
o3d.visualization.draw_geometries([pcd_down_20], window_name="Voxel 0.2")
print("Points (0.05):", len(np.asarray(pcd_down_05.points)))
print("Points (0.2):", len(np.asarray(pcd_down_20.points)))

Points (0.05): 90749
Points (0.2): 34625


In [8]:
print("Original points:", len(np.asarray(pcd.points)))
print("Downsampled points voxel 0.1:", len(np.asarray(pcd_down.points)))
print("Downsampled points voxel 0.2:", len(np.asarray(pcd_down_20.points)))
print("Downsampled points voxel 0.05:", len(np.asarray(pcd_down_05.points)))

Original points: 123397
Downsampled points voxel 0.1: 62162
Downsampled points voxel 0.2: 34625
Downsampled points voxel 0.05: 90749


# Remove Noise

## Code Filtering

In [9]:
# Statistical outlier removal
cl, ind = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Select inliers
pcd_filtered = pcd_down.select_by_index(ind)

# Visualize
o3d.visualization.draw_geometries([pcd_filtered], window_name="Filtered Point Cloud")

# Print stats
print("Filtered points:", len(np.asarray(pcd_filtered.points)))
print("Removed points:", len(np.asarray(pcd_down.points)) - len(np.asarray(pcd_filtered.points)))

Filtered points: 60139
Removed points: 2023


- **Filtered Point Cloud**  
  ![Filtered Point Cloud](filtered.png)

# Segment Ground vs. Objects

# Code Segmentation

In [10]:
# RANSAC ground segmentation
plane_model, inliers = pcd_filtered.segment_plane(
    distance_threshold=0.3, 
    ransac_n=3, 
    num_iterations=100
)
# Extract ground and obstacles
ground = pcd_filtered.select_by_index(inliers)
obstacles = pcd_filtered.select_by_index(inliers, invert=True)

# Color ground (green) and obstacles (red)
ground.paint_uniform_color([0, 1, 0]) # Green
obstacles.paint_uniform_color([1, 0, 0]) # Red

# Visualize
o3d.visualization.draw_geometries([ground, obstacles], window_name="Ground vs. Objects")

# Print stats
ground_points = len(np.asarray(ground.points))
obstacle_points = len(np.asarray(obstacles.points))
total_points = ground_points + obstacle_points
print("Ground points:", ground_points, f"({ground_points/total_points:.2%})")
print("Obstacle points:", obstacle_points)
print("Plane model:", plane_model)

Ground points: 29584 (49.19%)
Obstacle points: 30555
Plane model: [0.00227031 0.00497045 0.99998507 1.731208  ]


- **Ground vs. Objects**  
  ![Segmented Point Cloud](segmented.png)

# Cluster Objects

In [14]:
# Apply DBSCAN clustering
labels = np.array(obstacles.cluster_dbscan(eps=0.5, min_points=10))
max_label = labels.max()

# Color the clusters
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # Black for noise points
obstacles.colors = o3d.utility.Vector3dVector(colors[:, :3])

# Visualize the clustered obstacles
o3d.visualization.draw_geometries([obstacles], window_name="Clustered Objects")

print(f"Number of clusters: {max_label + 1} (excluding noise)")

Number of clusters: 212 (excluding noise)


# Label Objects

In [ ]:
labels_dict = {}
for i in range(max_label + 1):
    cluster = obstacles.select_by_index(np.where(labels == i)[0])
    num_points = len(np.asarray(cluster.points))
    labels_dict[i] = "Car" if num_points > 100 else "Pedestrian"
with open("labels.txt", "w") as f:
    for cluster_id, label in labels_dict.items():
        f.write(f"Cluster {cluster_id}: {label}\n")
print(labels_dict)

# Process a Sequence

In [ ]:
data_dir = "kitti/2011_09_26/0005/velodyne_points/data/"
frames = sorted(os.listdir(data_dir))[:5]
os.makedirs("output", exist_ok=True)
for frame in frames:
    points = np.fromfile(os.path.join(data_dir, frame), dtype=np.float32).reshape(-1, 4)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    pcd_down = pcd.voxel_down_sample(voxel_size=0.1)
    cl, ind = pcd_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    pcd_filtered = pcd_down.select_by_index(ind)
    plane_model, inliers = pcd_filtered.segment_plane(distance_threshold=0.3, ransac_n=3, num_iterations=100)
    obstacles = pcd_filtered.select_by_index(inliers, invert=True)
    labels = np.array(obstacles.cluster_dbscan(eps=0.5, min_points=10))
    colors = plt.get_cmap("tab20")(labels / (labels.max() if labels.max() > 0 else 1))
    obstacles.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.io.write_point_cloud(f"output/frame_{frame}.pcd", obstacles)
    print(f"Processed {frame}")